In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import warnings

warnings.filterwarnings('ignore')

2024-06-06 07:59:55.074543: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-06 07:59:55.394754: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-06 07:59:56.791241: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [29]:
def scaled_dot_product_attention(q, k, v, mask):
  print("Applying SELF ATTENTION:")
  print("-------------------------------------------------")
  
  matmul_qk = tf.matmul(q, k, transpose_b=True) 
  print("\nmultiplying QUERIE and KEY:")
  print(matmul_qk)

  dk = tf.cast(tf.shape(k)[-1], tf.float32)
  print("\nTaking the dk size:")
  print(dk)

  scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)
  print("\nStandarding the process:")
  print(scaled_attention_logits)

  if mask is not None:
    scaled_attention_logits += (mask * -1e9)

  attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1)  
  print("\nApplying SOFTMAX in the Standarded vlaues, CALCULATING attention weights:")
  print(attention_weights)
  output = tf.matmul(attention_weights, v)
  print("\nCalculating the outpu, multipling the attention weights and the VALUE MATRIX:")
  print(output)
  print("-------------------------------------------------")
  return output, attention_weights

In [35]:
y = tf.random.uniform((1, 6, 2))  # (batch_size, encoder_sequence, d_model)
y

<tf.Tensor: shape=(1, 6, 2), dtype=float32, numpy=
array([[[0.86278236, 0.30886674],
        [0.87680805, 0.03432608],
        [0.47888672, 0.74300647],
        [0.8201885 , 0.4476421 ],
        [0.5970541 , 0.7578428 ],
        [0.68740153, 0.8398106 ]]], dtype=float32)>

In [36]:
scaled_dot_product_attention(y, y, y, mask=None)

Applying SELF ATTENTION:
-------------------------------------------------

multiplying QUERIE and KEY:
tf.Tensor(
[[[0.8397921  0.7670967  0.642665   0.8459059  0.7492002  0.8524675 ]
  [0.7670967  0.7699706  0.44539624 0.7345137  0.54951566 0.6315466 ]
  [0.642665   0.44539624 0.7813911  0.72537833 0.8490034  0.9531722 ]
  [0.8459059  0.7345137  0.72537833 0.87309265 0.82893926 0.93973345]
  [0.7492002  0.54951566 0.8490034  0.82893926 0.9307993  1.0468603 ]
  [0.8524675  0.6315466  0.9531722  0.93973345 1.0468603  1.1778027 ]]], shape=(1, 6, 6), dtype=float32)

Taking the dk size:
tf.Tensor(2.0, shape=(), dtype=float32)

Standarding the process:
tf.Tensor(
[[[0.59382266 0.5424193  0.45443282 0.5981458  0.5297646  0.6027855 ]
  [0.5424193  0.5444514  0.31494272 0.5193796  0.38856626 0.4465709 ]
  [0.45443282 0.31494272 0.55252695 0.51291996 0.6003361  0.67399454]
  [0.5981458  0.5193796  0.51291996 0.6173698  0.58614856 0.6644919 ]
  [0.5297646  0.38856626 0.6003361  0.58614856 0.658

(<tf.Tensor: shape=(1, 6, 2), dtype=float32, numpy=
 array([[[0.7259617 , 0.519068  ],
         [0.7327624 , 0.5026113 ],
         [0.7101444 , 0.5525122 ],
         [0.7220472 , 0.52806944],
         [0.7115505 , 0.5506196 ],
         [0.7110103 , 0.5530305 ]]], dtype=float32)>,
 <tf.Tensor: shape=(1, 6, 6), dtype=float32, numpy=
 array([[[0.17327806, 0.16459607, 0.15073267, 0.1740288 , 0.16252626,
          0.17483813],
         [0.18044682, 0.18081388, 0.14373331, 0.17633691, 0.15471473,
          0.16395429],
         [0.1553829 , 0.1351523 , 0.17139767, 0.16474181, 0.17979111,
          0.19353418],
         [0.16895768, 0.15616016, 0.15515466, 0.17223714, 0.16694278,
          0.1805476 ],
         [0.15696792, 0.13629793, 0.1684456 , 0.16607265, 0.17847551,
          0.19374035],
         [0.15627827, 0.13367662, 0.16781248, 0.16622537, 0.17930616,
          0.19670102]]], dtype=float32)>)

In [51]:
class MultiHeadAttention(tf.keras.layers.Layer):
  def __init__(self, d_model, num_heads):
    super(MultiHeadAttention, self).__init__()
    self.num_heads = num_heads
    self.d_model = d_model

    assert d_model % self.num_heads == 0

    self.depth = d_model // self.num_heads

    self.wq = tf.keras.layers.Dense(d_model,use_bias=False)
    self.wk = tf.keras.layers.Dense(d_model,use_bias=False)
    self.wv = tf.keras.layers.Dense(d_model,use_bias=False)

    self.dense = tf.keras.layers.Dense(d_model)

  def split_heads(self, x, batch_size):
    """Split the last dimension into (num_heads, depth).
    Transpose the result such that the shape is (batch_size, num_heads, seq_len, depth)
    """
    x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))
    return tf.transpose(x, perm=[0, 2, 1, 3])

  def call(self, v, k, q, mask):
    batch_size = tf.shape(q)[0]
    print("Batch size = :{}".format(batch_size))
    q = self.wq(q)
    k = self.wk(k)  
    v = self.wv(v)
    print("Querie : \n")  
    print(q)
    print("\n Key : \n")  
    print(k)
    print("\n Value : \n")  
    print(v)

    q = self.split_heads(q, batch_size) 
    k = self.split_heads(k, batch_size)  
    v = self.split_heads(v, batch_size)

    print("\n Querie splited: \n")  
    print(q)  
    print("\n Key splited: \n")  
    print(k)
    print("\n Value splited: \n")  
    print(v)  

    scaled_attention, attention_weights = scaled_dot_product_attention(
        q, k, v, mask)

    scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3]) 
    print("\n Transposing the output self attention \n")
    print(scaled_attention)
    concat_attention = tf.reshape(scaled_attention,
                                  (batch_size, -1, self.d_model))  
    print("\n Concating the Transposed output self attention \n")
    print(concat_attention)
    output = self.dense(concat_attention)  
    print("\n Applying into a dense layer the Concat Transposed output self attention \n")
    print(output)
    return output, attention_weights

In [52]:
temp_mha = MultiHeadAttention(d_model=32, num_heads=2)

In [53]:
out, attn = temp_mha(y, k=y, q=y, mask=None)
out.shape, attn.shape

Batch size = :Tensor("strided_slice:0", shape=(), dtype=int32)
Querie : 

Tensor("dense_64_1/Add:0", shape=(1, 6, 32), dtype=float32)

 Key : 

Tensor("dense_65_1/Add:0", shape=(1, 6, 32), dtype=float32)

 Value : 

Tensor("dense_66_1/Add:0", shape=(1, 6, 32), dtype=float32)

 Querie splited: 

Tensor("transpose:0", shape=(1, 2, 6, 16), dtype=float32)

 Key splited: 

Tensor("transpose_1:0", shape=(1, 2, 6, 16), dtype=float32)

 Value splited: 

Tensor("transpose_2:0", shape=(1, 2, 6, 16), dtype=float32)
Applying SELF ATTENTION:
-------------------------------------------------

multiplying QUERIE and KEY:
Tensor("MatMul:0", shape=(1, 2, 6, 6), dtype=float32)

Taking the dk size:
Tensor("Cast:0", shape=(), dtype=float32)

Standarding the process:
Tensor("truediv:0", shape=(1, 2, 6, 6), dtype=float32)

Applying SOFTMAX in the Standarded vlaues, CALCULATING attention weights:
Tensor("Softmax:0", shape=(1, 2, 6, 6), dtype=float32)

Calculating the outpu, multipling the attention weights a

(TensorShape([1, 6, 32]), TensorShape([1, 2, 6, 6]))

In [54]:
class InterpretableMultiHeadSelfAttention(tf.keras.layers.Layer):
    def __init__(self, heads, units, dropout_rate):
        super().__init__()
        d_att = units // heads
        self._queries = tf.keras.layers.Dense(heads*d_att, use_bias=False)
        self._keys = tf.keras.layers.Dense(heads*d_att, use_bias=False)
        self._values = tf.keras.layers.Dense(d_att, use_bias=False)

        self._reshape = tf.keras.layers.TimeDistributed(tf.keras.layers.Reshape([heads, d_att]))
        self._softmax = tf.keras.layers.Softmax()
        
        self._attention = tf.keras.layers.Attention(dropout=dropout_rate)
        self._out_weights = tf.keras.layers.Dense(units, use_bias=False)

    def call(self, inputs):
        query, value, key = inputs, inputs, inputs

        query = self._reshape(self._queries(query))
        key = self._reshape(self._keys(key))

        value = tf.repeat(tf.expand_dims(self._values(value), -2), tf.shape(key)[-2], -2)

        outputs, attention = self._attention(
            [query/tf.sqrt(tf.cast(tf.shape(key)[-1], float)), value, key],
            use_causal_mask=True, return_attention_scores=True
        )

        outputs = self._out_weights(tf.reduce_mean(outputs, -2))
        attention = tf.reduce_mean(attention, -2)

        return outputs, attention

In [55]:
teste = InterpretableMultiHeadSelfAttention(2,32,0.0)

In [56]:
teste(y)

2024-06-06 09:27:13.189034: W tensorflow/core/framework/op_kernel.cc:1839] OP_REQUIRES failed at transpose_op.cc:142 : INVALID_ARGUMENT: transpose expects a vector of size 4. But input(1) is a vector of size 3
2024-06-06 09:27:13.189072: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: INVALID_ARGUMENT: transpose expects a vector of size 4. But input(1) is a vector of size 3


InvalidArgumentError: Exception encountered when calling Attention.call().

[1m{{function_node __wrapped__Transpose_device_/job:localhost/replica:0/task:0/device:GPU:0}} transpose expects a vector of size 4. But input(1) is a vector of size 3 [Op:Transpose][0m

Arguments received by Attention.call():
  • inputs=['tf.Tensor(shape=(1, 6, 2, 16), dtype=float32)', 'tf.Tensor(shape=(1, 6, 2, 16), dtype=float32)', 'tf.Tensor(shape=(1, 6, 2, 16), dtype=float32)']
  • mask=['None', 'None', 'None']
  • training=False
  • return_attention_scores=True
  • use_causal_mask=True